In [3]:
import cv2
import numpy as np
import os
import json

path = input('Enter a file path: ')  # path of the folder of the masked images
dirs = os.listdir(path)

for item in dirs:
    if os.path.isfile(os.path.join(path, item)):
        filename, ext = os.path.splitext(item)
        if ext.lower() == '.png':  # to process only BMP files
            print(item)  # name of the file
            image = cv2.imread(os.path.join(path, item))  # load the image

            cv2.waitKey(0)
            height, width, _ = image.shape  # height and width of the image

            # Grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Find Canny edges
            edged = cv2.Canny(gray, 100, 200)
            cv2.waitKey(0)

            # Finding Contours
            contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            print("Number of Contours found =", len(contours))

            # Filter contours based on shape (considering oval capsules)
            oval_contours = []
            for cnt in contours:
                perimeter = cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, 0.002 * perimeter, True) #pproximates polygon with another polygon with less vertices
                if len(approx) >= 6:  # Oval-like shape has at least 6 vertices
                    oval_contours.append(approx)

            print("Number of Oval Contours found =", len(oval_contours))

            # Annotate polygons
            annotations = []
            for cnt in oval_contours:
                annotation = []
                for point in cnt[:, 0, :]:
                    x, y = point
                    annotation.append([float(x), float(y)])
                annotations.append(annotation)

            json_path = os.path.join(path, filename + ".json")
            with open(json_path, "w+") as f:  # same filename as image
                # format of the json
                data = {
                    "version": "5.1.1",
                    "flags": {},
                    "shapes": [
                        {
                            "label": "product",
                            "points": annotation,
                            "group_id": None,
                            "shape_type": "polygon",
                            "flags": {}
                        } for annotation in annotations
                    ],
                    "imagePath": item,  # image name
                    "imageData": None,
                    "imageHeight": height,  # height of the image
                    "imageWidth": width  # width of the image
                }

                json.dump(data, f, indent=4)

Enter a file path: C:/Users/dhyan/Downloads/data/output_oval capsules/
pills.png
Number of Contours found = 9
Number of Oval Contours found = 9
